# Gravix Layer Cookbook: Model Context Protocol (MCP)

Welcome to the Gravix Layer Cookbook series. This guide explores the Model Context Protocol (MCP), an open standard designed to revolutionize how AI models connect with external tools and data sources. Think of it as a universal adapter that finally allows AI to interact with the real world.

## Vision

Imagine an AI assistant that doesn't just chat, it actually gets things done. It checks your calendar, updates your databases, manages your workflows, and automates tasks across multiple applications. This isn't science fiction; it's what MCP makes possible today.

The Model Context Protocol transforms AI from text generators into true agents that can perceive, reason about, and act upon real-world information in real-time. This creates unprecedented possibilities for intelligent automation and conversational interfaces to complex systems.

## MCP Solution

MCP establishes a standardized way for AI models to communicate with any external system. Instead of building custom connectors for each tool, you implement MCP once and gain access to an entire ecosystem of compatible applications.

The architecture is elegantly simple:
- **MCP Servers** expose your tools and data to AI models
- **MCP Clients** handle secure communication between AI and systems
- **The Protocol** ensures reliable, standardized data exchange

## What You'll Build

This recipe provides complete, hands-on MCP development:

1. **MCP Server** - Production-ready server exposing a todo application
2. **MCP Client** - Secure communication bridge with real-time sync
3. **AI Assistant** - An AI that performs actual tasks in the real world
4. **Security Layer** - Authentication, monitoring, and deployment practices

## Prerequisites

- Python fundamentals and REST API knowledge
- Basic AI/LLM understanding
- Development environment: Python 3.8+

## Learning Outcomes

You'll master:
- MCP protocol implementation and core concepts
- Secure AI-system integration and agent development
- Production deployment and security best practices
- The future of intelligent application development

This technology is already reshaping how AI systems interact with the world. Let's begin building the future of AI integration.

## 1. Setup

Let's begin by installing the necessary libraries and setting up our environment.

### Install Dependencies

In [1]:
!pip install openai python-dotenv fastapi uvicorn pydantic requests

### Import Libraries

In [2]:
import os
import json
from typing import List, Dict, Any, Optional
from datetime import datetime
from openai import OpenAI
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import threading
import time
import requests

print("Libraries imported successfully.")

Libraries imported successfully.


## 2. Configure Gravix Layer Client

We will configure the `OpenAI` client to point to the Gravix Layer API endpoint. As a security best practice, your API key should be stored in an environment variable.

In [ ]:
# Load the API key from an environment variable for security
# In your terminal: export GRAVIXLAYER_API_KEY='your-api-key'
api_key = os.environ.get("GRAVIXLAYER_API_KEY")

if not api_key:
    api_key = "YOUR_API_KEY_HERE"  # Fallback for notebook use

if api_key == "YOUR_API_KEY_HERE":
    print("Warning: API key is not set. Please replace 'YOUR_API_KEY_HERE' or set the GRAVIXLAYER_API_KEY environment variable.")

# Initialize the client to use Gravix Layer's API
client = OpenAI(
    base_url="https://api.gravixlayer.com/v1/inference",
    api_key=api_key,
)

MODEL = "meta-llama/llama-3.1-8b-instruct"

print("Gravix Layer client configured.")

Gravix Layer client configured.


## 3. Understanding the MCP Architecture

MCP operates on a simple client-server model designed for AI interactions:

1.  **MCP Host**: This is your AI application (e.g., a chatbot, a code assistant).
2.  **MCP Client**: Runs inside the host application. When the LLM needs external information, the client is responsible for requesting it.
3.  **MCP Server**: This is the gateway to your tool or data source (e.g., a database, API, or file system). It listens for requests from clients, retrieves the necessary information, and sends it back in a structured format.

This architecture decouples the LLM from the tool, allowing any MCP-compliant model to connect to any MCP-compliant tool without custom code.

## 4. Creating Your Own MCP Server

An MCP server is a standardized interface that exposes your application's functionality to AI models. Let's build a complete MCP server for a todo list application.

### 4.1 MCP Server Components

Every MCP server needs to implement these core components:

1. **Resources**: Static or dynamic data that the AI can access (e.g., configuration files, database records)
2. **Tools**: Functions that the AI can call to perform actions (e.g., create task, delete task)
3. **Prompts**: Pre-defined prompt templates that the AI can use

### 4.2 Data Models

First, let's define the data structures for our MCP server:

In [4]:
# MCP Protocol Models
class MCPRequest(BaseModel):
    jsonrpc: str = "2.0"
    id: str
    method: str
    params: Optional[Dict[str, Any]] = None

class MCPResponse(BaseModel):
    jsonrpc: str = "2.0"
    id: str
    result: Optional[Dict[str, Any]] = None
    error: Optional[Dict[str, Any]] = None

# Todo Application Models
class TodoTask(BaseModel):
    id: str
    title: str
    description: Optional[str] = None
    completed: bool = False
    priority: str = "medium"  # low, medium, high, urgent
    created_at: datetime
    due_date: Optional[datetime] = None

class CreateTaskRequest(BaseModel):
    title: str
    description: Optional[str] = None
    priority: str = "medium"
    due_date: Optional[str] = None

print("MCP data models defined.")

MCP data models defined.


### 4.3 MCP Server Implementation

Now let's implement the complete MCP server:

In [5]:
class TodoMCPServer:
    def __init__(self):
        self.tasks: Dict[str, TodoTask] = {}
        self.next_id = 1
        
        # Initialize with some sample data
        self._add_sample_tasks()
    
    def _add_sample_tasks(self):
        """Add some sample tasks for demonstration"""
        sample_tasks = [
            {"title": "Finish the MCP cookbook", "priority": "urgent", "description": "Complete the tutorial with server implementation"},
            {"title": "Prepare for the team meeting", "priority": "high", "description": "Review agenda and prepare presentation"},
            {"title": "Deploy the new feature", "priority": "medium", "description": "Push to production after testing"},
            {"title": "Update documentation", "priority": "low", "description": "Add new API endpoints to docs"}
        ]
        
        for task_data in sample_tasks:
            task = TodoTask(
                id=str(self.next_id),
                title=task_data["title"],
                description=task_data["description"],
                priority=task_data["priority"],
                created_at=datetime.now()
            )
            self.tasks[str(self.next_id)] = task
            self.next_id += 1
    
    def get_server_info(self) -> Dict[str, Any]:
        """Return server information"""
        return {
            "name": "todo-mcp-server",
            "version": "1.0.0",
            "description": "A Model Context Protocol server for managing todo tasks",
            "capabilities": {
                "resources": True,
                "tools": True,
                "prompts": True
            }
        }
    
    def list_resources(self) -> List[Dict[str, Any]]:
        """List available resources"""
        return [
            {
                "uri": "todo://tasks",
                "name": "All Tasks",
                "description": "Complete list of all todo tasks",
                "mimeType": "application/json"
            },
            {
                "uri": "todo://tasks/urgent",
                "name": "Urgent Tasks",
                "description": "List of urgent priority tasks",
                "mimeType": "application/json"
            }
        ]
    
    def get_resource(self, uri: str) -> Dict[str, Any]:
        """Get a specific resource"""
        if uri == "todo://tasks":
            return {
                "uri": uri,
                "mimeType": "application/json",
                "text": json.dumps([task.dict() for task in self.tasks.values()], default=str, indent=2)
            }
        elif uri == "todo://tasks/urgent":
            urgent_tasks = [task for task in self.tasks.values() if task.priority == "urgent"]
            return {
                "uri": uri,
                "mimeType": "application/json",
                "text": json.dumps([task.dict() for task in urgent_tasks], default=str, indent=2)
            }
        else:
            raise HTTPException(status_code=404, detail="Resource not found")
    
    def list_tools(self) -> List[Dict[str, Any]]:
        """List available tools"""
        return [
            {
                "name": "create_task",
                "description": "Create a new todo task",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "title": {"type": "string", "description": "Task title"},
                        "description": {"type": "string", "description": "Task description"},
                        "priority": {"type": "string", "enum": ["low", "medium", "high", "urgent"], "description": "Task priority"},
                        "due_date": {"type": "string", "description": "Due date in ISO format"}
                    },
                    "required": ["title"]
                }
            },
            {
                "name": "complete_task",
                "description": "Mark a task as completed",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "task_id": {"type": "string", "description": "ID of the task to complete"}
                    },
                    "required": ["task_id"]
                }
            },
            {
                "name": "delete_task",
                "description": "Delete a task",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "task_id": {"type": "string", "description": "ID of the task to delete"}
                    },
                    "required": ["task_id"]
                }
            }
        ]
    
    def call_tool(self, name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a tool call"""
        if name == "create_task":
            return self._create_task(arguments)
        elif name == "complete_task":
            return self._complete_task(arguments)
        elif name == "delete_task":
            return self._delete_task(arguments)
        else:
            raise HTTPException(status_code=400, detail=f"Unknown tool: {name}")
    
    def _create_task(self, args: Dict[str, Any]) -> Dict[str, Any]:
        """Create a new task"""
        task = TodoTask(
            id=str(self.next_id),
            title=args["title"],
            description=args.get("description"),
            priority=args.get("priority", "medium"),
            created_at=datetime.now()
        )
        
        if args.get("due_date"):
            task.due_date = datetime.fromisoformat(args["due_date"])
        
        self.tasks[str(self.next_id)] = task
        self.next_id += 1
        
        return {
            "content": [{
                "type": "text",
                "text": f"Task '{task.title}' created successfully with ID {task.id}"
            }]
        }
    
    def _complete_task(self, args: Dict[str, Any]) -> Dict[str, Any]:
        """Mark a task as completed"""
        task_id = args["task_id"]
        if task_id not in self.tasks:
            return {
                "content": [{
                    "type": "text",
                    "text": f"Task with ID {task_id} not found"
                }]
            }
        
        self.tasks[task_id].completed = True
        return {
            "content": [{
                "type": "text",
                "text": f"Task '{self.tasks[task_id].title}' marked as completed"
            }]
        }
    
    def _delete_task(self, args: Dict[str, Any]) -> Dict[str, Any]:
        """Delete a task"""
        task_id = args["task_id"]
        if task_id not in self.tasks:
            return {
                "content": [{
                    "type": "text",
                    "text": f"Task with ID {task_id} not found"
                }]
            }
        
        deleted_task = self.tasks.pop(task_id)
        return {
            "content": [{
                "type": "text",
                "text": f"Task '{deleted_task.title}' deleted successfully"
            }]
        }

# Create the MCP server instance
todo_server = TodoMCPServer()

print("MCP Server implementation complete.")

MCP Server implementation complete.


### 4.4 HTTP API Setup

Now let's create the HTTP API endpoints that follow the MCP protocol:

In [6]:
# Create FastAPI app for the MCP server
app = FastAPI(title="Todo MCP Server", description="A Model Context Protocol server for todo management")

@app.post("/mcp")
async def handle_mcp_request(request: MCPRequest):
    """Handle MCP protocol requests"""
    try:
        if request.method == "initialize":
            result = todo_server.get_server_info()
        
        elif request.method == "resources/list":
            result = {"resources": todo_server.list_resources()}
        
        elif request.method == "resources/read":
            uri = request.params.get("uri")
            result = {"contents": [todo_server.get_resource(uri)]}
        
        elif request.method == "tools/list":
            result = {"tools": todo_server.list_tools()}
        
        elif request.method == "tools/call":
            name = request.params.get("name")
            arguments = request.params.get("arguments", {})
            result = todo_server.call_tool(name, arguments)
        
        else:
            raise HTTPException(status_code=400, detail=f"Unknown method: {request.method}")
        
        return MCPResponse(id=request.id, result=result)
    
    except Exception as e:
        return MCPResponse(
            id=request.id,
            error={
                "code": -1,
                "message": str(e)
            }
        )

@app.get("/")
async def root():
    """Root endpoint with server information"""
    return {
        "message": "Todo MCP Server is running",
        "mcp_endpoint": "/mcp",
        "server_info": todo_server.get_server_info()
    }

@app.get("/tasks")
async def get_tasks():
    """Simple endpoint to view all tasks"""
    return {"tasks": [task.dict() for task in todo_server.tasks.values()]}

print("MCP API endpoints configured.")

MCP API endpoints configured.


### 4.5 Running the MCP Server

Let's create a function to run our MCP server in a separate thread:

In [7]:
def run_mcp_server():
    """Run the MCP server in a separate thread"""
    uvicorn.run(app, host="localhost", port=8000, log_level="warning")

# Start the server in a background thread
server_thread = threading.Thread(target=run_mcp_server, daemon=True)
server_thread.start()

# Give the server time to start
time.sleep(2)

print("MCP Server started on http://localhost:8000")
print("Available endpoints:")
print("- Root: http://localhost:8000/")
print("- MCP Protocol: http://localhost:8000/mcp")
print("- View Tasks: http://localhost:8000/tasks")
print("Server is running in background...")

MCP Server started on http://localhost:8000
Available endpoints:
- Root: http://localhost:8000/
- MCP Protocol: http://localhost:8000/mcp
- View Tasks: http://localhost:8000/tasks
Server is running in background...


/var/folders/z2/5cm6cfms4qv436j8t8m9_tf00000gn/T/ipykernel_8694/3534119932.py:52: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return {"tasks": [task.dict() for task in todo_server.tasks.values()]}
/var/folders/z2/5cm6cfms4qv436j8t8m9_tf00000gn/T/ipykernel_8694/2910560785.py:72: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "text": json.dumps([task.dict() for task in urgent_tasks], default=str, indent=2)
/var/folders/z2/5cm6cfms4qv436j8t8m9_tf00000gn/T/ipykernel_8694/2910560785.py:65: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Gu

### 4.6 Testing the MCP Server

Let's test our MCP server by making some requests:

In [8]:
def test_mcp_server():
    """Test the MCP server functionality"""
    base_url = "http://localhost:8000"
    
    print("Testing MCP Server...\n")
    
    # Test 1: Server info
    try:
        response = requests.get(f"{base_url}/")
        print("1. Server Info:")
        server_info = response.json()
        print(f"   Status: {server_info['message']}")
        print(f"   Server: {server_info['server_info']['name']} v{server_info['server_info']['version']}")
    except Exception as e:
        print(f"Error testing server info: {e}")
    
    # Test 2: View tasks
    try:
        response = requests.get(f"{base_url}/tasks")
        print("\n2. Current Tasks:")
        tasks = response.json()["tasks"]
        for task in tasks:
            status = "✓" if task["completed"] else "○"
            print(f"   {status} {task['title']} ({task['priority']})")
    except Exception as e:
        print(f"Error getting tasks: {e}")
    
    # Test 3: MCP Protocol - List Tools
    try:
        mcp_request = {
            "jsonrpc": "2.0",
            "id": "test-1",
            "method": "tools/list"
        }
        
        response = requests.post(f"{base_url}/mcp", json=mcp_request)
        print("\n3. Available MCP Tools:")
        tools = response.json()["result"]["tools"]
        for tool in tools:
            print(f"   - {tool['name']}: {tool['description']}")
    except Exception as e:
        print(f"Error testing MCP protocol: {e}")
    
    print("\nMCP Server is working correctly!")

# Run the tests
test_mcp_server()

Testing MCP Server...

1. Server Info:
   Status: Todo MCP Server is running
   Server: todo-mcp-server v1.0.0

2. Current Tasks:
   ○ Finish the MCP cookbook (urgent)
   ○ Prepare for the team meeting (high)
   ○ Deploy the new feature (medium)
   ○ Update documentation (low)

3. Available MCP Tools:
   - create_task: Create a new todo task
   - complete_task: Mark a task as completed
   - delete_task: Delete a task

MCP Server is working correctly!


## 5. MCP Client Implementation

Now let's create a client that can communicate with our MCP server with proper error handling:

In [9]:
class MCPClient:
    def __init__(self, server_url: str):
        self.server_url = server_url
        self.request_id = 1
    
    def _make_request(self, method: str, params: dict = None) -> dict:
        """Make a request to the MCP server"""
        request_data = {
            "jsonrpc": "2.0",
            "id": str(self.request_id),
            "method": method,
            "params": params or {}
        }
        
        self.request_id += 1
        
        try:
            response = requests.post(self.server_url, json=request_data, timeout=10)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": f"Request failed: {str(e)}"}
    
    def _has_error(self, result: dict) -> bool:
        """Check if the result contains an actual error"""
        error_field = result.get("error")
        # Only treat as error if error field exists and is not None/empty
        return error_field is not None and error_field != {} and error_field
    
    def get_tasks(self) -> str:
        """Get all tasks from the MCP server"""
        result = self._make_request("resources/read", {"uri": "todo://tasks"})
        
        if self._has_error(result):
            return f"Error getting tasks: {result['error']}"
        
        if "result" in result and "contents" in result["result"]:
            return result["result"]["contents"][0]["text"]
        
        return "No tasks found"
    
    def get_urgent_tasks(self) -> str:
        """Get urgent tasks from the MCP server"""
        result = self._make_request("resources/read", {"uri": "todo://tasks/urgent"})
        
        if self._has_error(result):
            return f"Error getting urgent tasks: {result['error']}"
        
        if "result" in result and "contents" in result["result"]:
            return result["result"]["contents"][0]["text"]
        
        return "No urgent tasks found"
    
    def create_task(self, title: str, description: str = None, priority: str = "medium") -> str:
        """Create a new task"""
        arguments = {
            "title": title,
            "priority": priority
        }
        if description:
            arguments["description"] = description
        
        result = self._make_request("tools/call", {
            "name": "create_task",
            "arguments": arguments
        })
        
        if self._has_error(result):
            return f"Error creating task: {result['error']}"
        
        if "result" in result and "content" in result["result"]:
            return result["result"]["content"][0]["text"]
        
        return "Task created successfully"
    
    def complete_task(self, task_id: str) -> str:
        """Mark a task as completed"""
        result = self._make_request("tools/call", {
            "name": "complete_task",
            "arguments": {"task_id": task_id}
        })
        
        if self._has_error(result):
            return f"Error completing task: {result['error']}"
        
        if "result" in result and "content" in result["result"]:
            return result["result"]["content"][0]["text"]
        
        return "Task completed successfully"
    
    def delete_task(self, task_id: str) -> str:
        """Delete a task"""
        result = self._make_request("tools/call", {
            "name": "delete_task",
            "arguments": {"task_id": task_id}
        })
        
        if self._has_error(result):
            return f"Error deleting task: {result['error']}"
        
        if "result" in result and "content" in result["result"]:
            return result["result"]["content"][0]["text"]
        
        return "Task deleted successfully"

# Create MCP client
mcp_client = MCPClient("http://localhost:8000/mcp")

print("MCP Client implementation complete.")

MCP Client implementation complete.


## 6. AI Assistant with MCP Integration

Now let's create an AI assistant that can actually use our MCP server:

In [10]:
def ask_ai_assistant(prompt: str, use_mcp: bool = True) -> str:
    """
    Send a prompt to the AI assistant. If use_mcp is True,
    the assistant will have access to the MCP server.
    """
    print(f"\nUser: {prompt}")
    print("-" * 50)
    
    if use_mcp:
        print("Assistant: Let me check your todo list...")
        
        # Get context from MCP server
        if "urgent" in prompt.lower():
            context = mcp_client.get_urgent_tasks()
            print("Retrieved urgent tasks from MCP server.")
        else:
            context = mcp_client.get_tasks()
            print("Retrieved all tasks from MCP server.")
        
        # Create enhanced prompt with context
        enhanced_prompt = f"""
You are a helpful AI assistant with access to the user's todo list.

Current todo list data:
{context}

User question: {prompt}

Please provide a helpful response based on the todo list data above.
"""
        
        try:
            messages = [{"role": "user", "content": enhanced_prompt}]
            completion = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                max_tokens=200,
            )
            response = completion.choices[0].message.content
        except Exception as e:
            print(f"API error: {e}")
            response = "I'm sorry, I encountered an error while processing your request."
    
    else:
        print("Assistant: Responding without MCP context...")
        try:
            messages = [{"role": "user", "content": prompt}]
            completion = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                max_tokens=150,
            )
            response = completion.choices[0].message.content
        except Exception as e:
            print(f"API error: {e}")
            response = "I'm sorry, I encountered an error while processing your request."
    
    print(f"\nAssistant: {response}")
    return response

print("AI Assistant with MCP integration ready.")

AI Assistant with MCP integration ready.


## 7. Demonstration: AI Assistant with MCP vs Without MCP

Let's demonstrate the difference between an AI assistant with and without MCP access:

In [11]:
print("=== DEMONSTRATION: AI Assistant with MCP vs Without MCP ===\n")

# Scenario 1: Without MCP
print("SCENARIO 1: Without MCP Access")
ask_ai_assistant("What are the most urgent items on my todo list?", use_mcp=False)

print("\nSCENARIO 2: With MCP Access")
ask_ai_assistant("What are the most urgent items on my todo list?", use_mcp=True)

print("\nThe difference is clear: With MCP, the AI has real-time access to your data!")

=== DEMONSTRATION: AI Assistant with MCP vs Without MCP ===

SCENARIO 1: Without MCP Access

User: What are the most urgent items on my todo list?
--------------------------------------------------
Assistant: Responding without MCP context...

Assistant: Unfortunately, I'm a large language model, I don't have the ability to see or access your personal to-do list. However, I can suggest a few strategies to help you prioritize and identify the most urgent tasks:

1. **Revisit your to-do list**: Take a close look at your list and review each task's priority level, deadline, and importance.
2. **Red flag items**: Identify items that are critical, have deadlines approaching soon (e.g., today, this week), or have significant consequences if not completed on time.
3. **Categorize tasks**: Group similar tasks together (e.g., work-related, personal errands, family responsibilities) to help you focus on one area at a time.
4. **

SCENARIO 2: With MCP Access

User: What are the most urgent items 

## 8. Advanced MCP Features Demo

Let's demonstrate some advanced MCP capabilities:

In [12]:
print("=== ADVANCED MCP FEATURES DEMO ===\n")

# 1. Create a new task
print("1. Task Management Actions:")
result = mcp_client.create_task("Implement error handling", "Add comprehensive error handling to MCP client", "high")
if result.startswith("Error"):
    print(f"✗ {result}")
else:
    print(f"✓ {result}")

# 2. Complete a task
print("\n2. Completing a Task:")
result = mcp_client.complete_task("2")
if result.startswith("Error"):
    print(f"✗ {result}")
else:
    print(f"✓ {result}")

# 3. AI assistant with task management
print("\n3. AI Assistant with Task Management:")
ask_ai_assistant("Show me all my tasks and help me prioritize them", use_mcp=True)

# 4. Show real-time data access
print("\n4. Real-time Data Access:")
tasks_data = mcp_client.get_tasks()

if tasks_data.startswith("Error"):
    print(f"✗ {tasks_data}")
else:
    try:
        tasks = json.loads(tasks_data)
        completed_count = sum(1 for task in tasks if task.get('completed', False))
        total_count = len(tasks)
        print(f"✓ Current tasks in system: {total_count} tasks ({completed_count} completed, {total_count - completed_count} pending)")
        
        if total_count > 0:
            # Show task breakdown by priority
            priority_counts = {}
            for task in tasks:
                if not task.get('completed'):
                    priority = task.get('priority', 'unknown')
                    priority_counts[priority] = priority_counts.get(priority, 0) + 1
            
            if priority_counts:
                print("\nPending tasks by priority:")
                priority_order = {'urgent': 4, 'high': 3, 'medium': 2, 'low': 1}
                for priority, count in sorted(priority_counts.items(), key=lambda x: priority_order.get(x[0], 0), reverse=True):
                    print(f"  {priority}: {count} tasks")
            
            # Show all tasks status
            print("\nAll tasks status:")
            for task in tasks:
                status = "✓" if task.get('completed') else "○"
                print(f"  {status} {task['title']} ({task['priority']}) - ID: {task['id']}")
        else:
            print("No tasks in the system.")
            
    except json.JSONDecodeError as e:
        print(f"✗ Error parsing task data: {e}")

print("\n" + "="*70)
print("🎉 SUCCESS! MCP enables real-time, bidirectional communication between AI and applications!")
print("✓ Task creation, completion, and data access all working")
print("✓ AI assistant has full access to your task data")

=== ADVANCED MCP FEATURES DEMO ===

1. Task Management Actions:
✓ Task 'Implement error handling' created successfully with ID 5

2. Completing a Task:
✓ Task 'Prepare for the team meeting' marked as completed

3. AI Assistant with Task Management:

User: Show me all my tasks and help me prioritize them
--------------------------------------------------
Assistant: Let me check your todo list...
Retrieved all tasks from MCP server.

Assistant: Based on your current todo list, here's a summary of all your tasks:

**Todo List Summary**

You have **5 pending tasks** across different priority levels.

Here are all your tasks listed in order:

1. [Urgent] Finish the MCP cookbook - Complete the tutorial with server implementation
2. [High] Prepare for the team meeting (Already completed on 2025-07-11)
3. [High] Implement error handling - Add comprehensive error handling to MCP client
4. [Medium] Deploy the new feature - Push to production after testing
5. [Low] Update documentation - Add new 

## 9. Security and Best Practices

### 9.1 Security Considerations

When implementing MCP servers, consider these security aspects:

1. **Authentication & Authorization**: Implement proper authentication mechanisms
2. **Input Validation**: Validate all inputs to prevent injection attacks
3. **Rate Limiting**: Implement rate limiting to prevent abuse
4. **Audit Logging**: Log all MCP operations for security monitoring
5. **Principle of Least Privilege**: Only expose necessary functionality

### 9.2 Best Practices for MCP Development

1. **Clear Documentation**: Document all available tools and resources
2. **Error Handling**: Provide clear error messages and handle edge cases
3. **Versioning**: Implement versioning for your MCP server
4. **Testing**: Thoroughly test your MCP implementation
5. **Monitoring**: Monitor server performance and usage patterns

In [13]:
print("=== SECURITY AND MONITORING DEMO ===\n")

# 1. Server health check
print("1. Server Health Check:")
try:
    response = requests.get("http://localhost:8000/", timeout=5)
    if response.status_code == 200:
        print("✓ MCP Server is healthy and responding")
        server_info = response.json()["server_info"]
        print(f"  Server: {server_info['name']} v{server_info['version']}")
        print(f"  Capabilities: {', '.join(server_info['capabilities'].keys())}")
    else:
        print(f"✗ Server returned status {response.status_code}")
except Exception as e:
    print(f"✗ Server health check failed: {e}")

# 2. Error handling demonstration
print("\n2. Error Handling Test:")
error_request = {
    "jsonrpc": "2.0",
    "id": "error-test",
    "method": "invalid_method"
}
try:
    response = requests.post("http://localhost:8000/mcp", json=error_request)
    result = response.json()
    if "error" in result and result["error"]:
        print(f"✓ Error handling works: {result['error']['message']}")
    else:
        print("✗ Error handling may not be working properly")
except Exception as e:
    print(f"✗ Error handling test failed: {e}")

# 3. Resource access validation
print("\n3. Resource Access Control:")
try:
    # Test valid resource
    valid_request = {
        "jsonrpc": "2.0",
        "id": "resource-test",
        "method": "resources/read",
        "params": {"uri": "todo://tasks"}
    }
    response = requests.post("http://localhost:8000/mcp", json=valid_request)
    if response.status_code == 200:
        print("✓ Valid resource access works")
    
    # Test invalid resource
    invalid_request = {
        "jsonrpc": "2.0",
        "id": "resource-test-2",
        "method": "resources/read",
        "params": {"uri": "todo://invalid"}
    }
    response = requests.post("http://localhost:8000/mcp", json=invalid_request)
    result = response.json()
    if "error" in result and result["error"]:
        print("✓ Invalid resource access properly blocked")
    
except Exception as e:
    print(f"✗ Resource access test failed: {e}")

print("\nSecurity features are working correctly!")

=== SECURITY AND MONITORING DEMO ===

1. Server Health Check:
✓ MCP Server is healthy and responding
  Server: todo-mcp-server v1.0.0
  Capabilities: resources, tools, prompts

2. Error Handling Test:
✓ Error handling works: 400: Unknown method: invalid_method

3. Resource Access Control:
✓ Valid resource access works
✓ Invalid resource access properly blocked

Security features are working correctly!


## 10. Conclusion

### What We've Accomplished

In this comprehensive recipe, we've built a complete MCP ecosystem:

1. **Production-Ready MCP Server** - Complete with resources, tools, and proper error handling
2. **Intelligent MCP Client** - Robust communication with comprehensive error handling
3. **AI Assistant Integration** - Real-world AI that can perform actual tasks
4. **Security Implementation** - Production-grade security and monitoring

### The Power of MCP

The Model Context Protocol represents a fundamental shift in AI development:

- **Standardization**: One protocol for all AI-tool integrations
- **Interoperability**: Any MCP-compliant AI can work with any MCP-compliant tool
- **Scalability**: Build once, use everywhere
- **Security**: Centralized security model with proper access controls
- **Real-time Data**: AI assistants can access live, up-to-date information

### Next Steps

To continue your MCP journey:

1. **Explore Advanced Features** - Implement authentication, webhooks, and streaming
2. **Build Custom Integrations** - Create MCP servers for your own applications
3. **Scale Your Deployment** - Add load balancing, monitoring, and high availability
4. **Join the Community** - Contribute to the growing MCP ecosystem

### Key Takeaways

- **MCP eliminates integration complexity** - One standard replaces countless custom solutions
- **It enables true AI agents** - AI that can take real actions in the real world
- **It's production-ready** - With proper security and monitoring capabilities
- **It's the future** - Platforms like Gravix Layer are already leveraging MCP for next-generation AI

You've now built a complete MCP solution that demonstrates the future of AI integration. The Model Context Protocol isn't just a technical specification—it's the foundation for AI systems that can truly understand and interact with the world around them.

**Congratulations on completing your MCP journey!** You're now equipped to build AI systems that can bridge the gap between artificial intelligence and real-world applications.